In [1]:
import pandas as pd
import numpy as np

In [2]:
file_k = r"D:\ryano\OneDrive\Documents\Abschlussprojekt_Python\kunden.csv"
file_b = r"D:\ryano\OneDrive\Documents\Abschlussprojekt_Python\besucher.csv"
file_geo = r"D:\ryano\OneDrive\Documents\Abschlussprojekt_Python\geo.txt"

In [3]:
df_k = pd.read_csv(file_k, sep = ";", decimal = ".")
df_b = pd.read_csv(file_b, sep = ";", decimal = ",")
df_geo = pd.read_table("geo.txt", delimiter = "\t") #\t muss verwendet werden damit die Spalten der Liste aus der txt Datei separiert werden können

In [4]:
df_geo

,KundeNr,Niederlassung
0,K0001,Sachsen
1,K0002,NRW
2,K0003,Hessen
3,K0004,Bayern
4,K0005,Bayern
...,...,...
1990,K1991,Hessen
1991,K1992,Baden-Württemberg
1992,K1993,Nordrhein-Westfalen
1993,K1994,Berlin


In [5]:
df_b_k = pd.concat([df_b,df_k], ignore_index = True)
df_b_k

,Alter,Einkommen,Geschlecht,Zeit,KundeNr,Preis
0,30,39919.095335,0.0,29.0,K1814,NaN
1,41,53430.673086,0.0,27.0,K1544,NaN
2,64,73163.216927,0.0,32.0,K1945,NaN
3,41,36761.482581,0.0,44.0,K1825,NaN
4,35,42684.331632,1.0,33.0,K1144,NaN
...,...,...,...,...,...,...
1990,70,69581.000000,1.0,51.0,K0467,103751.58
1991,38,48301.000000,1.0,33.0,K0122,64230.98
1992,67,63547.000000,1.0,41.0,K1045,103123.57
1993,37,32686.000000,1.0,36.0,K1096,49215.87


In [6]:
df_bkg = pd.merge(df_b_k, df_geo, on = "KundeNr")
df_bkg

,Alter,Einkommen,Geschlecht,Zeit,KundeNr,Preis,Niederlassung
0,30,39919.095335,0.0,29.0,K1814,NaN,Bayern
1,41,53430.673086,0.0,27.0,K1544,NaN,Baden-Württemberg
2,64,73163.216927,0.0,32.0,K1945,NaN,Baden-Württemberg
3,41,36761.482581,0.0,44.0,K1825,NaN,Baden-Württemberg
4,35,42684.331632,1.0,33.0,K1144,NaN,Nordrhein-Westfalen
...,...,...,...,...,...,...,...
1990,70,69581.000000,1.0,51.0,K0467,103751.58,Thüringen
1991,38,48301.000000,1.0,33.0,K0122,64230.98,Niedersachsen
1992,67,63547.000000,1.0,41.0,K1045,103123.57,Berlin
1993,37,32686.000000,1.0,36.0,K1096,49215.87,Baden-Württemberg


In [7]:
# Berechnung von Q1 (25. Perzentil), Q3 (75. Perzentil) und IQR
Q1 = df_bkg['Einkommen'].quantile(0.25)
Q3 = df_bkg['Einkommen'].quantile(0.75)
IQR = Q3 - Q1

# Berechnung der Ausreißer-Grenzen
untere_grenze = Q1 - 1.6 * IQR #Faktor auf die Ausreißer nach eigenem Ermessen angepasst
obere_grenze = Q3 + 5 * IQR #Faktor auf die Ausreißer nach eigenem Ermessen angepasst

# Identifizierung von Ausreißern
df_bkg['Ausreißer'] = (df_bkg['Einkommen'] < untere_grenze) | (df_bkg['Einkommen'] > obere_grenze)

df_bkg

,Alter,Einkommen,Geschlecht,Zeit,KundeNr,Preis,Niederlassung,Ausreißer
0,30,39919.095335,0.0,29.0,K1814,NaN,Bayern,False
1,41,53430.673086,0.0,27.0,K1544,NaN,Baden-Württemberg,False
2,64,73163.216927,0.0,32.0,K1945,NaN,Baden-Württemberg,False
3,41,36761.482581,0.0,44.0,K1825,NaN,Baden-Württemberg,False
4,35,42684.331632,1.0,33.0,K1144,NaN,Nordrhein-Westfalen,False
...,...,...,...,...,...,...,...,...
1990,70,69581.000000,1.0,51.0,K0467,103751.58,Thüringen,False
1991,38,48301.000000,1.0,33.0,K0122,64230.98,Niedersachsen,False
1992,67,63547.000000,1.0,41.0,K1045,103123.57,Berlin,False
1993,37,32686.000000,1.0,36.0,K1096,49215.87,Baden-Württemberg,False


In [8]:
untere_grenze

9204.719763225141

In [9]:
#Ausreißer bzgl. des Einkommens identifizieren
filtered_df_bkg = df_bkg[df_bkg["Ausreißer"] == True]
filtered_df_bkg

,Alter,Einkommen,Geschlecht,Zeit,KundeNr,Preis,Niederlassung,Ausreißer
1299,68,1.000000e+09,0.0,52.0,K0701,87209.72,Nordrhein-Westfalen,True
1857,69,0.000000e+00,1.0,65.0,K1021,125140.05,Bayern,True
1973,59,-6.284037e+04,0.0,46.0,K1083,92837.59,Baden-Württemberg,True


In [10]:
#Median des Einkommens bestimmen 
einkommen_medianwert = df_bkg["Einkommen"].median()
einkommen_medianwert 

59104.51301372704

In [11]:
#Überprüfung des Median
Q2 = df_bkg['Einkommen'].quantile(0.5)
Q2

59104.51301372704

In [12]:
#Ersetzen der oberen und unteren Werte
df_bkg.loc[df_bkg["Einkommen"] < untere_grenze, "Einkommen"] = einkommen_medianwert
df_bkg.loc[df_bkg["Einkommen"] > obere_grenze, "Einkommen"] = einkommen_medianwert

In [13]:
df_bkg.iloc[1299]

Alter                             68
Einkommen               59104.513014
Geschlecht                       0.0
Zeit                            52.0
KundeNr                        K0701
Preis                       87209.72
Niederlassung    Nordrhein-Westfalen
Ausreißer                       True
Name: 1299, dtype: object

In [14]:
df_bkg.iloc[1857, 1]

59104.51301372704

In [15]:
#Identifikation fehlender Werte bei Kategorie Geschlecht
geschlecht_NAN = df_bkg[df_bkg["Geschlecht"].isna()]
geschlecht_NAN

,Alter,Einkommen,Geschlecht,Zeit,KundeNr,Preis,Niederlassung,Ausreißer
1001,41,46542.0,NaN,40.0,K0024,63477.00,Baden-Württemberg,False
1203,53,49798.0,NaN,39.0,K0166,61329.51,Nordrhein-Westfalen,False
1479,64,83452.0,NaN,62.0,K0090,133180.70,Niedersachsen,False
1503,52,66602.0,NaN,45.0,K0325,98756.07,Bayern,False
1622,67,91461.0,NaN,66.0,K0790,131748.07,Berlin,False


In [16]:
#Identifikation des Modalwert bei der Kategorie Geschlecht
geschlecht_modalwert = df_bkg["Geschlecht"].mode()[0]
geschlecht_modalwert 

1.0

In [17]:
#Geschlechtswerte ohne Eintrag durch Modalwert ersetzen
df_bkg["Geschlecht"] = df_bkg["Geschlecht"].fillna(geschlecht_modalwert)
df_bkg

,Alter,Einkommen,Geschlecht,Zeit,KundeNr,Preis,Niederlassung,Ausreißer
0,30,39919.095335,0.0,29.0,K1814,NaN,Bayern,False
1,41,53430.673086,0.0,27.0,K1544,NaN,Baden-Württemberg,False
2,64,73163.216927,0.0,32.0,K1945,NaN,Baden-Württemberg,False
3,41,36761.482581,0.0,44.0,K1825,NaN,Baden-Württemberg,False
4,35,42684.331632,1.0,33.0,K1144,NaN,Nordrhein-Westfalen,False
...,...,...,...,...,...,...,...,...
1990,70,69581.000000,1.0,51.0,K0467,103751.58,Thüringen,False
1991,38,48301.000000,1.0,33.0,K0122,64230.98,Niedersachsen,False
1992,67,63547.000000,1.0,41.0,K1045,103123.57,Berlin,False
1993,37,32686.000000,1.0,36.0,K1096,49215.87,Baden-Württemberg,False


In [18]:
#Identifikation von wahrscheinlich falschen oder gar illegalen Altersangaebn
unglaubwürdiges_alter = df_bkg[(df_bkg["Alter"] > 90) | (df_bkg["Alter"] < 18)]
unglaubwürdiges_alter

,Alter,Einkommen,Geschlecht,Zeit,KundeNr,Preis,Niederlassung,Ausreißer
1033,100,88749.0,0.0,64.0,K0494,119134.23,Bayern,False
1602,120,61857.0,1.0,52.0,K0039,104214.07,Baden-Württemberg,False


In [19]:
#Identifikation des Modalwert bei der Kategorie Alter
alter_medianwert = df_bkg["Alter"].median()
alter_medianwert 

51.0

In [20]:
#Bereinigung Alter und Überprüfung
df_bkg.loc[df_bkg["Alter"] > 90, "Alter"] = alter_medianwert
df_bkg.iloc[1602]

Alter                           51
Einkommen                  61857.0
Geschlecht                     1.0
Zeit                          52.0
KundeNr                      K0039
Preis                    104214.07
Niederlassung    Baden-Württemberg
Ausreißer                    False
Name: 1602, dtype: object

In [21]:
#Definition der Bundesländer
bundesländer = ["Baden-Württemberg", "Bayern", "Berlin", "Brandenburg", "Bremen", "Hamburg", "Hessen",
                "Mecklenburg-Vorpommern", "Niedersachsen", "Nordrhein-Westfalen", "Rheinland-Pfalz", "Saarland",
                "Sachsen", "Sachsen-Anhalt", "Schleswig-Holstein", "Thüringen"]
#Nur jene Einträge für Niederlassungen anzeigen lassen, welche *nicht* in vorheriger Liste vorhanden sind
bundesländer_falsche_namen = df_bkg[~df_bkg["Niederlassung"].isin(bundesländer)]
bundesländer_falsche_namen

,Alter,Einkommen,Geschlecht,Zeit,KundeNr,Preis,Niederlassung,Ausreißer
299,44,54025.601631,1.0,30.0,K1987,NaN,NRW,False
491,49,54754.525450,1.0,44.0,K1995,NaN,Berlin-Charlottenburg,False
587,63,84772.480847,1.0,30.0,K1973,NaN,Düsseldorf,False
840,32,40776.023145,1.0,13.0,K1609,NaN,BERLIN,False
911,43,52813.000000,0.0,44.0,K0762,67106.32,BERLIN,False
1249,61,56063.000000,1.0,44.0,K0679,98148.54,Berlin-Mitte,False
1472,55,66738.000000,1.0,52.0,K0413,120215.50,Düsseldorf,False
1510,62,75575.000000,1.0,50.0,K0023,124516.56,NRW,False
1783,58,65271.000000,1.0,44.0,K0002,106258.99,NRW,False
1812,30,28784.000000,1.0,34.0,K0677,42416.07,Berlin-Mitte,False


In [22]:
#Jetzt ersetzen wir die ungewollten Bezeichnungen der Niederlassungen mit den gewollten Bezeichnungen
#1. den "falschen" Schlüsseln die gewollten Werte zuordnen
ersetzen_dict = {
    "Berlin-Charlottenburg": "Berlin",
    "BERLIN": "Berlin",
    "Berlin-Mitte": "Berlin",
    "Düsseldorf": "Nordrhein-Westfalen",
    "NRW": "Nordrhein-Westfalen",
}
#2. Die geändert Werte jetzt in das Datafram übertragen
df_bkg["Niederlassung"].replace(ersetzen_dict, inplace=True)

In [30]:
#Überprüfung ob korrekt ersetzt wurde (Stichprobe)
df_bkg.iloc[1472, 6]

'Nordrhein-Westfalen'